# Full Tutorial

This notebook aims to show all the features of `panama`, and especially how they interplay.

## Installation
Installing `panama` is easy, just run 
```bash
pip install corsika-panama
```
Or install it by cloning, so you also have access to the install script for CORSIKA7.
```bash
git clone git@github.com:The-Ludwig/PANAMA.git
cd PANAMA
pip install -e .
```

### CORSIKA7
For this notebook to make sense, you need a CORSIKA7 installation (preferably with the `EHIST` option). 
If you cloned the `panama` repository in the last step, you can easily download and install a pre-configured version (EHIST, SIBYLL2.3d, CURVED, URQDM) of CORSIKA7. You need to have a FORTRAN compiler installed:
```
CORSIKA_VERSION=77500 CORSIKA_PW= admin/download_corsika.sh
```
This will install CORSIKA7 in the folder `corsika-77500`. You oviously need the CORSIKA_PW, which you will get by mail, after requesting to be on the CORSIKA7 mailing list, [as described on their website here](https://www.iap.kit.edu/corsika/79.php).

## Generating a CORSIKA7 Muon Simulation
`panama` provides two ways of executing CORSIKA7 (in parallel). You can either use the Command Line Interface (CLI), available through `panama run` after installation, or you can do it inside python via an `CorsikaRunner` object.
The CLI itself is only a small wrapper around `panama.CorsikaRunner`. This tutorial focuses on the use of `CorsikaRunner`, but translating the code to a `panama` command, is straightforeward. For that, have a look at `panama run --help`.

### Explanation of modified CORSIKA cards

In [1]:
import panama as pn

In [2]:
?pn.get_weights

Signature:
pn.get_weights(
    df_run: 'pd.DataFrame',
    df_event: 'pd.DataFrame',
    df: 'pd.DataFrame',
    model: 'CosmicRayFlux' = <fluxcomp.cosmic_ray_fluxes.H3a object at 0x7f8a7eb94910>,
    proton_only: 'bool' = False,
    groups: 'dict[PDGID, tuple[int, int]] | None' = None,
) -> 'pd.DataFrame'
Docstring:
Adds the column "weight" too df_particle to reweight for given primary flux.

Parameters
----------
df_run: The run dataframe (as returned by `read_corsika_particle_files_to_dataframe`)
df_event: The event dataframe (as returned by `read_corsika_particle_files_to_dataframe`)
df: The particle dataframe (as returned by `read_corsika_particle_files_to_dataframe`)
model: The Cosmic Ray primary flux model (instance of CRFlux)
proton_only: If set to true (default), only proton pdgid weights are non-zero and refer to
    all-nucleon flux.
groups: The elements in the model (values: Tuple[Zmin, Zmax]) associated with the MC-primary

Returns
-------
weights: A dataframe with the wei